<a href="https://colab.research.google.com/github/nvvishnu/CS6910-Assignments/blob/main/Assignment%203/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import csv
import tensorflow as tf
from tensorflow import keras
from keras.utils.vis_utils import plot_model
!pip install wandb
import wandb
from wandb.keras import WandbCallback 
import pandas as pd
!pip install dataframe-image
import dataframe_image as dfi

  Using cached https://files.pythonhosted.org/packages/98/5f/45439b4767334b868e1c8c35b1b0ba3747d8c21be77b79f09eed7aa3c72b/wandb-0.10.30-py2.py3-none-any.whl
     |████████████████████████████████| 133kB 2.9MB/s 
  Using cached https://files.pythonhosted.org/packages/fd/01/ff260a18caaf4457eb028c96eeb405c4a230ca06c8ec9c1379f813caa52e/configparser-5.0.2-py3-none-any.whl
     |████████████████████████████████| 174kB 13.9MB/s 
  Using cached https://files.pythonhosted.org/packages/32/c8/564be4d12629b912ea431f1a50eb8b3b9d00f1a0b1ceff17f266be190007/subprocess32-3.5.4.tar.gz
  Using cached https://files.pythonhosted.org/packages/f5/e8/f6bd1eee09314e7e6dee49cbe2c5e22314ccdb38db16c9fc72d2fa80d054/docker_pycreds-0.4.0-py2.py3-none-any.whl
     |████████████████████████████████| 71kB 8.0MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=db33978bf90d8931b846cb60c837ef9b2c7bc60b55bf8353ee7d5d1254d77e3c
  Stored in directory: /root/.cache/pip/wheels/0b/04

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Train data preprocessing

In [ ]:
datapath = "/content/drive/MyDrive/Assignment3/hi/lexicons/hi.translit.sampled.train.tsv" #add path here after mounting drive 
input_texts = []
target_texts = []
expected_texts = []
input_characters = set([' '])
target_characters = set([' '])
f = open(datapath)
lines = csv.reader(f,delimiter='\t')
 
for row in lines:
    input_text = row[1]
    target_text = row[0]
    #adding '\t' as start and '\n' as stop
    expected_texts.append(target_text)
    target_text = "\t"+target_text+"\n"
    
 
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
 
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
 
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)
 
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
 
encoder_input_data = np.zeros((len(input_texts),max_encoder_seq_length),dtype='float32')
decoder_input_data = np.zeros((len(input_texts),max_decoder_seq_length),dtype='float32')
decoder_target_data = np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')
 
 
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    
    for t,char in enumerate(input_text):
       encoder_input_data[i,t] = input_token_index[char]
 
    encoder_input_data[i,t+1:] = input_token_index[' ']
 
    for t,char in enumerate(target_text):
        decoder_input_data[i,t] = target_token_index[char]
        if(t>0):
          decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    
    decoder_input_data[i,t+1:] = target_token_index[' ']
    decoder_target_data[i,t:,target_token_index[' ']] = 1.0

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

# Val data Preprocessing


In [ ]:
val_datapath = "/content/drive/MyDrive/Assignment3/hi/lexicons/hi.translit.sampled.dev.tsv"
val_f = open(val_datapath)
lines = csv.reader(val_f,delimiter='\t')
val_input_texts = []
val_expected_texts = []
for row in lines:
    input_text = row[1]
    target_text = row[0]
    #adding '\t' as start and '\n' as stop
    val_expected_texts.append(target_text)
    val_input_texts.append(input_text)

max_val_encoder_seq_length = max([len(txt) for txt in val_input_texts])

val_encoder_input_data = np.zeros((len(val_input_texts),max_val_encoder_seq_length),dtype='float32')

for i,input_text in enumerate(val_input_texts):
    
    for t,char in enumerate(input_text):
       val_encoder_input_data[i,t] = input_token_index[char]
 
    val_encoder_input_data[i,t+1:] = input_token_index[' ']


# Test data processing


In [ ]:
test_datapath = '/content/drive/MyDrive/Assignment3/hi/lexicons/hi.translit.sampled.test.tsv'
test_f = open(test_datapath)
lines = csv.reader(test_f,delimiter='\t')
test_input_texts = []
test_expected_texts = []
for row in lines:
    input_text = row[1]
    target_text = row[0]
    #adding '\t' as start and '\n' as stop
    test_expected_texts.append(target_text)
    test_input_texts.append(input_text)

max_test_encoder_seq_length = max([len(txt) for txt in test_input_texts])

test_encoder_input_data = np.zeros((len(test_input_texts),max_test_encoder_seq_length),dtype='float32')

for i,input_text in enumerate(test_input_texts):
    
    for t,char in enumerate(input_text):
       test_encoder_input_data[i,t] = input_token_index[char]
 
    test_encoder_input_data[i,t+1:] = input_token_index[' ']


# Model definitions

In [ ]:
def get_model(num_encoder_tokens,num_decoder_tokens,embedding_size,num_encoder_layers,num_decoder_layers,hidden_layer_size,cell_type,drop_out):
    
    if(cell_type == 'LSTM'):
        rnn_cell = tf.keras.layers.LSTM
    elif(cell_type ==  'RNN'):
        rnn_cell = tf.keras.layers.SimpleRNN
    elif(cell_type == "GRU"):
        rnn_cell = tf.keras.layers.GRU
        

    encoder_inputs = tf.keras.Input(shape = (None,))
    encoder_out = tf.keras.layers.Embedding(input_dim = num_encoder_tokens,output_dim = embedding_size,name="encoderembedding")(encoder_inputs) 
  

    for i in range(num_encoder_layers):
        encoder_out,*encoder_states = rnn_cell(hidden_layer_size,return_sequences=True,return_state=True,dropout = drop_out,
                                               name="encoder_"+str(i))(encoder_out)

    decoder_inputs = tf.keras.Input(shape = (None,))
    decoder_out = tf.keras.layers.Embedding(input_dim = num_decoder_tokens,output_dim=embedding_size,name="decoderembedding")(decoder_inputs)
    
    for i in range(num_decoder_layers):
        decoder_out,*_ = rnn_cell(hidden_layer_size,return_sequences=True,return_state=True,dropout = drop_out,
                                  name="decoder_"+str(i))(decoder_out,initial_state = encoder_states)
    
    decoder_dense = tf.keras.layers.Dense(num_decoder_tokens, activation="softmax",name="decoderdense")
    decoder_outputs = decoder_dense(decoder_out)

    model = tf.keras.Model([encoder_inputs,decoder_inputs], decoder_outputs)

    model.compile(
      optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
    )
     
    # "Inference Model"
    encoder_inputs = model.input[0]

    enocoder_output,*encoder_states = model.get_layer("encoder_"+str(num_encoder_layers-1)).output
    encoder_model = tf.keras.Model(encoder_inputs,encoder_states)

    decoder_inputs = model.input[1]
    decoder_out = model.get_layer("decoderembedding")(decoder_inputs)
    
    decoder_state_inputs = []
    decoder_state_outputs = []
    
    for i in range(num_decoder_layers):
      
      curr_layer = model.get_layer("decoder_"+str(i))
      if (cell_type == "LSTM"):
        decoder_in = [tf.keras.Input(shape=(encoder_states[0].shape[1],)),tf.keras.Input(shape=(encoder_states[0].shape[1],))]
      else:
        decoder_in = [tf.keras.Input(shape=(encoder_states[0].shape[1],))]
      
      decoder_state_inputs += decoder_in

      decoder_out,*decoder_states = curr_layer(decoder_out,initial_state=decoder_in)

      decoder_state_outputs += decoder_states
    
    dense_layer = model.get_layer('decoderdense')
    decoder_output = dense_layer(decoder_out)

    decoder_model = tf.keras.Model([decoder_inputs]+decoder_state_inputs,[decoder_output]+decoder_state_outputs)
    

    return [model,encoder_model,decoder_model]

# Beam search Inference

In [ ]:

def fn(sequence):
    
    decoded_sentence = ""
    for i in sequence:
      sampled_char = reverse_target_char_index[i]
      if sampled_char == '\n':
         break
      decoded_sentence += sampled_char

    return decoded_sentence


def decode_sequence(input_seq,beam_size,encoder_model,decoder_model):
    
    num_samples = input_seq.shape[0]

    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)   

    target_seq = np.zeros((num_samples,1))
    # # Populate the first character of target sequence with the start character.
    target_seq[:,0] = target_token_index["\t"]

    num_decoder_layers = 0

    for layer in decoder_model.layers:
        if layer.name.startswith('decoder_') :
           num_decoder_layers += 1
    
    #Shape (None,list(list))
    sequence_list = [[list()] for i in range(num_samples)]
    
    #shape (None,curr_beamsize)
    sequence_score = np.empty(shape=(num_samples,1),dtype=object)
    sequence_score.fill(0.0)
    
    if(type(states_value) != list):
      states_value = [states_value]
    
    sequence_state = [states_value*num_decoder_layers]

    
    num_states = len(sequence_state[0])
    #print("num_states",num_states)
   
    #intially curr_beam size 1 after 1st prediction we set it back to beam_size
    k = 1

    for ii in range(max_decoder_seq_length):
        
        # shape (None,list)
        all_candidates = [ [] for i in range(num_samples) ]

        for x in range(k):
            
            #seq is (None,curr_seq)  curr_seq means list found till now
            seq = [sequence_list[i][x] for i in range(num_samples)]

            #score is (None,1) it is score of all samples xth sequence
            score = sequence_score[:,x]

            #curr_state is the state_corresponding to xth sequence for all samples 
            curr_state = sequence_state[x]

            if(ii>0):
                 #filling the target_seq with previous elements of this xth sequence for all samples 
                 target_seq = np.array([seq[i][ii-1] for i in range(num_samples)])
                 target_seq = target_seq.reshape((num_samples,1))
            
            #calling the predict function and storing the output_tokens and states
            output_tokens,*states= decoder_model.predict([target_seq] + curr_state)
            
            #for all samples updating the corresponding all_candidate list
            for i in range(num_samples):
              #all_candidates[i] is list of [seq_list,score,states] , at the end we sort this all_candidates list based on score
              all_candidates[i] += [[seq[i]+[j],score[i]- np.math.log(output_tokens[i,0,j]),[states[ii][i] for ii in range(num_states)]] for j in range(output_tokens.shape[2])]

        #setting k to beam size    
        k = beam_size

        for i in range(num_samples):    
           #sorting the all_candidates[i] list based on score
           all_candidates[i] = sorted(all_candidates[i],key=lambda tup:tup[1])[:k]
        
        # Now remake sequence_list,sequence_score and sequence_state
        sequence_list = [[all_candidates[i][j][0] for j in range(k)] for i in range(num_samples)]
        sequence_score = np.array([[all_candidates[i][j][1] for j in range(k)] for i in range(num_samples)])
        sequence_state = [ [ np.array([all_candidates[j][i][2][kk] for j in range(num_samples)]) for kk in range(num_states)] for i in range(k)]

    final_sequence = [sequence_list[i][0] for i in range(num_samples)]
    decoded_sentence = [fn(final_sequence[i]) for i in range(num_samples)]
    
    return decoded_sentence

# Custom call_back

In [ ]:
def f(a,b):
    cnt = 0
    for i in range(min(len(b),len(a))):
        if(a[i] == b[i]):
          cnt += 1
    cnt /= len(b)
    return cnt

class CustomCallback(keras.callbacks.Callback):
    
    def __init__(self, encoder_model,decoder_model,beam_size):
        self.model = model
        self.encoder_model = encoder_model
        self.decoder_model = decoder_model
        self.beam_size = beam_size

    def on_epoch_end(self, epoch, logs=None):
      
        if(epoch%2 == 1):
          x = decode_sequence(val_encoder_input_data,self.beam_size,self.encoder_model,self.decoder_model)
          word2word = (np.sum(np.array(x) == np.array(val_expected_texts))/len(x))
          charbychar = np.sum(np.array([f(a,b) for a,b in zip(x,val_expected_texts)]))/len(x)
          print("val_accuracy_word2word is: ",word2word)
          print("val_accuracy_char_by_char is: ", charbychar)
          wandb.log({'val_accuracy_word2word':word2word})
          wandb.log({'val_accuracy_charbychar':charbychar})

# Training code

In [ ]:
def train():
    config_default = { 
        'embedding_size' : 256,
        'num_encoder_layers': 2,
        'num_decoder_layers': 2,
        'hidden_layer_size': 256,
        'cell_type': 'LSTM',
        'drop_out': 0.25,
        'beam_size': 5
    }
    print("wand about to be initialised")
    wandb.init(config=config_default,project='Assignment3')
    config = wandb.config
    model,encoder_model,decoder_model = get_model(num_encoder_tokens,num_decoder_tokens,config.embedding_size,
                      config.num_encoder_layers,config.num_decoder_layers,config.hidden_layer_size,
                      config.cell_type,config.drop_out)
    model.fit(
      [encoder_input_data, decoder_input_data],
      decoder_target_data,
      batch_size=64,
      epochs=10,
      callbacks=[CustomCallback(encoder_model=encoder_model,decoder_model=decoder_model,beam_size=config.beam_size),WandbCallback()]
    )


In [ ]:
#sweep configuration
sweep_config = {
    'name': 'Recurrent_Neural_Networks',
    'method' : 'bayes',
    'metric': {        
         'goal': 'maximize',
         'name' : 'val_accuracy_word2word'
    },
    'parameters': {
        'embedding_size' : {
            'values': [64, 256,512]
        },
        'num_encoder_layers': {
            'values': [1,2,3]
        },
        'num_decoder_layers': {
            'values': [1,2,3]
        },
        'hidden_layer_size': {
            'values': [64, 256,512]
        },
        'cell_type': {
            'values': [ 'RNN', 'GRU', 'LSTM']
        },
        'drop_out': {
            'values' : [0.25,0.3]
        },
        'beam_size': {
            'values': [5]
        }
    }
}

In [ ]:
# sweep_id = wandb.sweep(sweep_config,entity="shivaram_22",project="Assignment3")

In [ ]:
# wandb.agent(sweep_id,project="Assignment3",function=train)

# Training Model with found best hyperparameters


In [ ]:
# Best hyperparameters from the wandb sweep are 
# beam_size = 5
# cell type = 'LSTM'
# drop_out = 0.3
# embedding_size = 512
# hidden_layer_size = 512
# num_encoder_layers = 2
# num decoder_layers = 2
model,encoder_model,decoder_model = get_model(num_encoder_tokens,num_decoder_tokens,512,2,2,512,'LSTM',0.3)
model.fit(
      [encoder_input_data, decoder_input_data],
      decoder_target_data,
      batch_size=64,
      epochs=10,
    )

In [ ]:
# helper function used for calculating charbyaccuracy
def f(a,b):
    cnt = 0
    for i in range(min(len(b),len(a))):
        if(a[i] == b[i]):
          cnt += 1
    cnt /= len(b)
    return cnt

In [ ]:
x = decode_sequence(val_encoder_input_data,5,encoder_model,decoder_model)
word2word = (np.sum(np.array(x) == np.array(val_expected_texts))/len(x))
charbychar = np.sum(np.array([f(a,b) for a,b in zip(x,val_expected_texts)]))/len(x)
print("val_accuracy_word2word is: ",word2word)
print("val_accuracy_char_by_char is: ", charbychar)
x = decode_sequence(test_encoder_input_data,5,encoder_model,decoder_model)
word2word = (np.sum(np.array(x) == np.array(test_expected_texts))/len(x))
charbychar = np.sum(np.array([f(a,b) for a,b in zip(x,test_expected_texts)]))/len(x)
print("test_accuracy_word2word is: ",word2word)
print("test_accuracy_char_by_char is: ", charbychar)

In [ ]:
vanilla_df = pd.DataFrame()
vanilla_df['input_texts'] = test_input_texts
vanilla_df['predictions'] = x
vanilla_df['actual'] = test_expected_texts
vanilla_df.to_csv("/content/drive/MyDrive/Assignment3/Vanilla_Predictions.tsv", sep="\t")

vanilla_random_sample = vanilla_df.sample(n = 10,replace=False)

In [ ]:
vanilla_random_sample.head(10)

# With Attention

In [ ]:
import tensorflow as tf
import os
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K

#Reference: 
# https://github.com/thushv89/attention_keras/blob/master/src/layers/attention.py


class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state
            inputs: (batchsize * 1 * de_in_dim)
            states: (batchsize * 1 * de_latent_dim)
            """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [ ]:
def model_with_attention(num_encoder_tokens,num_decoder_tokens,embedding_size,hidden_layer_size,cell_type,drop_out):
     
     if(cell_type == 'LSTM'):
        rnn_cell = tf.keras.layers.LSTM
     elif(cell_type ==  'RNN'):
        rnn_cell = tf.keras.layers.SimpleRNN
     elif(cell_type == "GRU"):
        rnn_cell = tf.keras.layers.GRU
    
     encoder_inputs = tf.keras.Input(shape = (None,))
     encoder_embedding_out = tf.keras.layers.Embedding(input_dim = num_encoder_tokens,output_dim = embedding_size,name="encoderembedding")(encoder_inputs)

     encoder_out,*encoder_states = rnn_cell(hidden_layer_size,return_sequences=True,return_state=True,dropout = drop_out,
                                               name="encoder_0")(encoder_embedding_out)

     decoder_inputs = tf.keras.Input(shape = (None,))
     decoder_embedding_out = tf.keras.layers.Embedding(input_dim = num_decoder_tokens,output_dim=embedding_size,name="decoderembedding")(decoder_inputs)
     
     decoder_out,*_ = rnn_cell(hidden_layer_size,return_sequences=True,return_state=True,dropout = drop_out,
                                  name="decoder_0")(decoder_embedding_out,initial_state = encoder_states)
    
     attn_layer = AttentionLayer(name='attention_layer')
     attn_out, attn_states = attn_layer([encoder_out, decoder_out])

     final_out = tf.keras.layers.Concatenate()([attn_out,decoder_out])
     decoder_dense = tf.keras.layers.Dense(num_decoder_tokens, activation="softmax",name="decoderdense")
     decoder_outputs = decoder_dense(final_out)

     model = tf.keras.Model([encoder_inputs,decoder_inputs], decoder_outputs)

     return model
